In [2]:
import tensorflow as tf

In [3]:
tf.version

<module 'tensorflow._api.v2.version' from '/usr/local/lib/python3.6/dist-packages/tensorflow/_api/v2/version/__init__.py'>

In [4]:
%tensorflow_version 2.x

In [5]:
import tensorflow_datasets as tfds
import tensorflow_hub as hub

In [6]:
tfds.list_builders()

['abstract_reasoning',
 'aeslc',
 'aflw2k3d',
 'amazon_us_reviews',
 'arc',
 'bair_robot_pushing_small',
 'beans',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'cos_e',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'diabetic_retinopathy_detection',
 'div2k',
 'dmlab',
 'downsampled_imagenet',
 'dsprites',
 'dtd',
 'duke_ultrasound',
 'dummy_dataset_shared_generator',
 'dummy_mnist',
 'emnist',
 'eraser_multi_rc',
 'esnli',
 'eurosat',
 'fashion_mnist',
 'flic',
 'flores',
 'food101',
 'gap',
 'gigaword',
 'glue',
 'gr

In [7]:
# Split the training set into 60% and 40%, so we'll end up with 15,000 examples
# for training, 10,000 examples for validation and 25,000 examples for testing.
train_data, validation_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True)

Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAPYNLS/imdb_reviews-train.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAPYNLS/imdb_reviews-test.tfrecord


Shuffling and writing examples to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0.incompleteAPYNLS/imdb_reviews-unsupervised.tfrecord


Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [8]:
type(train_data)

tensorflow.python.data.ops.dataset_ops.DatasetV1Adapter

In [9]:
train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))   # taking 10batch as sample
train_examples_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [10]:
train_labels_batch # targetted values for above 10 example batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [11]:
embed = hub.load("https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1")   # Using tensorflow hub model
embeddings = embed(["cat is on the mat", "dog is in the fog"])
embeddings    # Following model is use for conversion from word to vector

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[ 0.8666395 ,  0.35917717,  0.00579667,  0.681002  , -0.54226625,
         0.22343187, -0.38796625,  0.62195706,  0.22117122, -0.48538068,
        -1.2674141 ,  0.886369  , -0.3284907 , -0.13924702, -0.53327686,
         0.5739708 , -0.05905761,  0.13629246, -1.1718255 , -0.31494334],
       [ 0.960218  ,  0.62520486,  0.06261905,  0.37425604,  0.24782333,
        -0.39351934, -0.7418429 ,  0.56599647, -0.26197797, -0.69016844,
        -0.76565284,  0.71412426, -0.4537978 , -0.50701594, -0.8499377 ,
         0.8917156 , -0.30278975,  0.2149126 , -1.1098894 , -0.46719775]],
      dtype=float32)>

In [12]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True) # Creating hub layer where input shap is unknown and data type is string
hub_layer(train_examples_batch[:1])

<tf.Tensor: shape=(1, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ]],
      dtype=float32)>

In [13]:
from tensorflow import keras
from tensorflow.keras import layers

In [14]:
hub_layer = hub.KerasLayer("https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1", output_shape=[20],input_shape=[], dtype=tf.string,trainable=True)

model = keras.Sequential()
model.add(hub_layer) # adding hub layer of 20 output shape
model.add(keras.layers.Dense(16, activation='relu')) # hidddn layer of 16 neuron
model.add(keras.layers.Dense(1, activation='sigmoid')) # Output layer of 1 neuron

model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 20)                389380    
_________________________________________________________________
dense (Dense)                (None, 16)                336       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 389,733
Trainable params: 389,733
Non-trainable params: 0
_________________________________________________________________


In [15]:
model.compile(optimizer='adam',                                 # Compiling the model                 
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [16]:
model.fit(train_data.shuffle(10000).batch(512),epochs=20,validation_data=validation_data.batch(512)) 
# Training the model where shuffle=10000,batch=512,epochs/iteration=20,validation batch=512. Each iteration it will fine tune the model

Epoch 1/20
30/30 [==============================] - 3s 114ms/step - loss: 0.6904 - accuracy: 0.5012 - val_loss: 0.6837 - val_accuracy: 0.5072
Epoch 2/20
30/30 [==============================] - 3s 111ms/step - loss: 0.6719 - accuracy: 0.5737 - val_loss: 0.6605 - val_accuracy: 0.6229
Epoch 3/20
30/30 [==============================] - 3s 110ms/step - loss: 0.6539 - accuracy: 0.6513 - val_loss: 0.6481 - val_accuracy: 0.6719
Epoch 4/20
30/30 [==============================] - 3s 110ms/step - loss: 0.6400 - accuracy: 0.6963 - val_loss: 0.6358 - val_accuracy: 0.7149
Epoch 5/20
30/30 [==============================] - 3s 113ms/step - loss: 0.6253 - accuracy: 0.7419 - val_loss: 0.6237 - val_accuracy: 0.7578
Epoch 6/20
30/30 [==============================] - 3s 113ms/step - loss: 0.6104 - accuracy: 0.7820 - val_loss: 0.6096 - val_accuracy: 0.7876
Epoch 7/20
30/30 [==============================] - 3s 111ms/step - loss: 0.5961 - accuracy: 0.8205 - val_loss: 0.5999 - val_accuracy: 0.8086
Epoch 

In [17]:
results = model.evaluate(test_data.batch(512), verbose=20)     

for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

loss: 0.572
accuracy: 0.859


In [19]:
model.predict(["This is the worst movie I have ever seen",
               "It was not that bad",
               "It makes me depressed",
               "I feel high after watching DARK"])   # Value closer to 1 mean positive sentiment

array([[0.02197129],
       [0.00152036],
       [0.86550486],
       [0.26241276]], dtype=float32)